<a href="https://colab.research.google.com/github/QaziSaim/Transformer-Text-Generation/blob/main/Biggest_Transformer_Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install kagglehub

In [2]:
from google.colab import userdata
import os
os.environ['KAGGLE_API_KEY'] = userdata.get('kaggle_api_key')
os.environ['KAGGLE_USERNAME'] = userdata.get('kaggle_username')

### Downloading The Dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dhruvildave/en-fr-translation-dataset")

print("Path to dataset files:", path)

100%|██████████| 2.54G/2.54G [00:22<00:00, 122MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2


In [4]:
%ls /root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2
# %ls /kaggle/input/en-fr-translation-dataset/en-fr.csv

en-fr.csv


In [5]:
import pandas as pd
# path = '/kaggle/input/en-fr-translation-dataset/en-fr.csv'
path = '/root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2/en-fr.csv'
df = pd.read_csv('/root/.cache/kagglehub/datasets/dhruvildave/en-fr-translation-dataset/versions/2/en-fr.csv')
# df = pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')

In [21]:
df.dropna(inplace=True)

In [22]:
# df.drop_duplicates(inplace=True)

In [6]:
# en_file = open('english.txt','w',encoding='utf-8')
# fr_file = open('french.txt','w',encoding='utf-8')

# for chunk in pd.read_csv(path, chunksize=10000):
#   for en, fr in zip(chunk['en'],chunk['fr']):
#     if isinstance(en,str) and isinstance(fr,str):
#       en_file.write(en.strip() + "\n")
#       fr_file.write(fr.strip() + "\n")
# en_file.close()
# fr_file.close()

In [6]:
%%capture
%pip install sentencepiece


### Not Good takes lots of time

In [1]:
# import sentencepiece as spm
# spm.SentencePieceTrainer.Train(
#     input='english.txt,french.txt',
#     model_prefix = 'enfr',
#     vocab_size=16000,
#     character_coverage=1.0,
#     model_type='bpe'
# )

In [6]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
from tokenizers.normalizers import NFD, Lowercase, StripAccents
from tokenizers.pre_tokenizers import Whitespace

In [7]:
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence([NFD(), Lowercase(), StripAccents()])
tokenizer.pre_tokenizer = Whitespace()

In [8]:
trainer = trainers.BpeTrainer(vocab_size=32000,
                              min_frequency=2,
                              special_tokens=['<pad>','<unk>','<s>','</s>'])

In [24]:
def batch_iterator():
    for en, fr in zip(df["en"].tolist(), df["fr"].tolist()):
        yield en
        yield fr

In [25]:
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)


In [26]:
tokenizer.save('bpe_tokenizer.json')

In [ ]:
# class translation